In [1]:
from map_tool_box.modules import Data_Structure
from map_tool_box.modules import Environment
from map_tool_box.modules import Data_Map
from map_tool_box.modules import Control
from map_tool_box.modules import Spawner
from map_tool_box.modules import Astar
from map_tool_box.modules import Utils
from map_tool_box.modules import Model
from IPython.display import HTML
from pathlib import Path

In [2]:
# point to model you wish to evaluate
models_directory = Utils.get_global('models_directory') # check to make sure this is correct on your local computer (it should be auto)
model_name = 'AirSim_Navigation DRL_alpha' # change to proper model name if needed (name is also sub-directory path)
model_subdir = model_name.replace(' ', '/')
model_directory = Path(models_directory, model_subdir)

In [3]:
# read control params and make objects from config.py file
config_path = Path(model_directory, 'config.py')
%run $config_path

In [4]:
# read model from file
model_path = Path(model_directory, 'model.zip')
model = Model.read_model(model_path)

# access pytorch model
model.q_net

QNetwork(
  (features_extractor): CombinedExtractor(
    (extractors): ModuleDict(
      (img): NatureCNN(
        (cnn): Sequential(
          (0): Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
          (1): ReLU()
          (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
          (3): ReLU()
          (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
          (5): ReLU()
          (6): Flatten(start_dim=1, end_dim=-1)
        )
        (linear): Sequential(
          (0): Linear(in_features=25088, out_features=256, bias=True)
          (1): ReLU()
        )
      )
      (vec): Flatten(start_dim=1, end_dim=-1)
    )
  )
  (q_net): Sequential(
    (0): Linear(in_features=268, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=17, bias=True)
  )
)

In [5]:
# set parameters for which set of Astar paths to evaluate on
set_name = 'test' # train val test
n_paths = 100 # if None then will read all paths from file, otherwise an integer value specifying number of paths PER DIFFICULTY
difficulties = [i for i in range(1, 37)] # if None then will read all difficulties from file, otherwise expects a list of difficulty keys

# read paths from file (usese some variables read in from config.py file) 
    # -- you can overwrite map_name or astar_version, but the default values or those used to train the model
paths = Astar.read_curriculum(map_name, astar_version, set_name, n_paths, difficulties)

# create spawner object to iterate through paths from environment
spawner = Spawner.CurricululmEval(paths)

# create environment that we will step through (uses some objects read in from config.py file)
environment = Environment.Episodic(data_map, spawner, actor, observer, terminators)

In [6]:
# run through an example episode (view map_tool_box.modules.Environment.py to see what is saved in states)
states = Control.play_episode(environment, model, save_additional_state_info=True, save_observations=True)

# make animation from episodic states 
    # to see what is stored in state variables, view this function in map_tool_box.modules.Environment.Episodic.animate_episode()
animation = environment.animate_episode(states, show_rewards=False)

# show animation in notebook
display(HTML(animation.to_jshtml()))

# save as gif
#animation.save('episode.gif', writer='pillow', fps=2)

# view returned states from example episode
states

[{'step': 0,
  'action_value': 'start',
  'point': x:134 y:-228 z:4 d:1,
  'target': x:166 y:-226 z:4,
  'path_steps': 1,
  'observations': {'vec': array([0.21316154, 0.5589408 , 0.47411764, 0.4       , 0.        ,
          0.        , 0.        , 0.        , 0.        , 0.        ,
          0.        , 0.        ], dtype=float32),
   'img': array([[[  9,   7,   7, ..., 255, 255, 255],
           [  9,   9,   7, ..., 255, 255, 255],
           [  9,   9,   7, ..., 255, 255, 255],
           ...,
           [ 11,  11,  11, ...,  11,  11,  11],
           [ 11,  11,  10, ...,  10,  11,  11],
           [ 10,  10,  10, ...,  10,  10,  10]],
   
          [[  0,   0,   0, ...,   0,   0,   0],
           [  0,   0,   0, ...,   0,   0,   0],
           [  0,   0,   0, ...,   0,   0,   0],
           ...,
           [  0,   0,   0, ...,   0,   0,   0],
           [  0,   0,   0, ...,   0,   0,   0],
           [  0,   0,   0, ...,   0,   0,   0]],
   
          [[  0,   0,   0, ...,   0,   

# evaluate full set of paths

In [ ]:
# run entire evaluation over set astar paths and model
# turned flag off for collecting observations since it is memory heavy
episodes_states, accuracy = Control.eval(environment, model, save_additional_state_info=True, save_observations=True)

# accuracy is auto calculated for you, however you can iterate through the states to recalculate the same accuracy for when state['end']=='Goal'
print(f'accuracy = {accuracy*100:.2f}')